# Neural Network

## Load the Data

In [1]:
import numpy as np
import pandas as pd


train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

## Preprocessing

In [2]:
from preprocessing import preprocessing


X_train, Y_train, X_test = preprocessing(train_data, test_data)

print("X_train.shape =", X_train.shape)
print("Y_train.shape =", Y_train.shape)
print("X_test.shape =", X_test.shape)

X_train.shape = (891, 12)
Y_train.shape = (891, 1)
X_test.shape = (418, 12)
